In [25]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image

files = np.load("99%_filenames.npy")
training_labels = np.load("99%_label.npy")
arrshape = (3150, 96, 24, 3)
imgshape = (96, 24, 3)
imgmean = (96, 24)

mypath=os.path.dirname('speech_units/')
data = []
for fname in files:
    pathname = os.path.join(mypath, fname)
    img = Image.open(pathname)
    img = img.resize((96, 24))
    img = np.asarray(img)
    data.append(img)


In [26]:
training_images = np.asarray(data)
training_images = training_images.reshape(arrshape)
training_images=training_images / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=imgshape),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(42, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=20)
# test_loss, test_accuracy = model.evaluate(test_images, test_labels)
# print ('Test loss: {}, Test accuracy: {}'.format(test_loss, test_accuracy*100))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 94, 22, 64)        1792      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 47, 11, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 45, 9, 64)         36928     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 22, 4, 64)         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 5632)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               721024    
_________________________________________________________________
dense_21 (Dense)             (None, 42)              